In [2]:
observation_bones_list = [(0,11),
                    (0,12),
                    
                    (11,13),
                    (13,15),
                    (15,17),
                    (15,19),
                    (15,21),
                    (17,19),
                    
                    (12,14),
                    (14,16),
                    (16,18),
                    (16,20),
                    (16,22),
                    (18,20),
                    
                    (11,12),
                    (11,23),
                    (12,24),
                    (23,24),
                    
                    (23,25),
                    (25,27),
                    (27,29),
                    (27,31),
                    (29,31),
                    
                    (24,26),
                    (26,28),
                    (28,30),
                    (28,32),
                    (30,32)]

In [72]:
observation_bones_list = [(23,25),(24,26)]

## basic

In [15]:
import csv
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2


def read_keypoints_from_csv(csv_file):
    keypoints = []
    world_keypoints = []
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            frame_keypoints = []
            frame_world_keypoints = []
            for i in range(33):
                x = float(row[1 + i*4])
                y = float(row[2 + i*4])
                z = float(row[3 + i*4])
                visibility = float(row[4 + i*4])
                frame_keypoints.append(landmark_pb2.NormalizedLandmark(x=x, y=y, z=z, visibility=visibility))
                
                world_x = float(row[133 + i*4])
                world_y = float(row[134 + i*4])
                world_z = float(row[135 + i*4])
                world_visibility = float(row[136 + i*4])
                frame_world_keypoints.append(landmark_pb2.NormalizedLandmark(x=world_x, y=world_y, z=world_z, visibility=world_visibility))
                
            keypoints.append(frame_keypoints)
            world_keypoints.append(frame_world_keypoints)
    return keypoints, world_keypoints

def extract_frame(video_path, frame_index):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        raise ValueError(f"Could not read frame {frame_index} from {video_path}")
    return frame

def draw_landmarks(image, landmarks, color):
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    pose_landmarks = landmark_pb2.NormalizedLandmarkList(landmark=landmarks)
    mp_drawing.draw_landmarks(image, pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2))
    return image

def clamp(value, min_value, max_value):
    return max(min_value, min(value, max_value))

def get_closest_points_on_segments(A, B, C, D):
    """
    Calculate the closest points on two line segments in 3D space.
    
    Parameters:
    A, B, C, D (np.array): Points defining the two line segments (A-B and C-D).
    
    Returns:
    tuple: Closest points on the two line segments.
    """
    AB = B - A
    CD = D - C
    AC = A - C
    
    AB_dot_CD = np.dot(AB, CD)
    AB_dot_AB = np.dot(AB, AB)
    CD_dot_CD = np.dot(CD, CD)
    AC_dot_AB = np.dot(AC, AB)
    AC_dot_CD = np.dot(AC, CD)
    
    denom = AB_dot_AB * CD_dot_CD - AB_dot_CD * AB_dot_CD
    if denom == 0:
        # Segments are parallel, handle separately if needed
        return None, None
    
    s = (AB_dot_CD * AC_dot_CD - CD_dot_CD * AC_dot_AB) / denom
    t = (AB_dot_AB * AC_dot_CD - AB_dot_CD * AC_dot_AB) / denom
    
    # Clamp s and t to the range [0, 1] to ensure the points are on the segments
    s = clamp(s, 0, 1)
    t = clamp(t, 0, 1)
    
    P = A + s * AB
    Q = C + t * CD
    
    return P, Q

def get_distance_two_segments(A, B, C, D):
    """
    Calculate the shortest distance between two line segments in 3D space.
    
    Parameters:
    A, B, C, D (np.array): Points defining the two line segments (A-B and C-D).
    
    Returns:
    float: The shortest distance between the two line segments.
    """
    P, Q = get_closest_points_on_segments(A, B, C, D)
    if P is None or Q is None:
        # Segments are parallel, handle separately if needed
        return np.linalg.norm(np.cross(A - C, B - A)) / np.linalg.norm(B - A)
    
    return np.linalg.norm(P - Q)

def compare_vector_and_visualize(video_file1, video_file2, csv_file1, csv_file2, frame1_index, frame2_index, output_image, distance_threshold=0.04):
    keypoints1, world_keypoints1 = read_keypoints_from_csv(csv_file1)
    keypoints2, world_keypoints2 = read_keypoints_from_csv(csv_file2)

    frame1_7 = extract_frame(video_file1, frame1_index)
    frame2_9 = extract_frame(video_file2, frame2_index)

    highlight_indices = []
    for bone in observation_bones_list:

        bone_start_idx = bone[0]
        bone_finish_idx = bone[1]
        
        A = np.array([world_keypoints1[frame1_index][bone_start_idx].x, world_keypoints1[frame1_index][bone_start_idx].y, world_keypoints1[frame1_index][bone_start_idx].z])
        B = np.array([world_keypoints1[frame1_index][bone_finish_idx].x, world_keypoints1[frame1_index][bone_finish_idx].y, world_keypoints1[frame1_index][bone_finish_idx].z])
        
        C = np.array([world_keypoints2[frame2_index][bone_start_idx].x, world_keypoints2[frame2_index][bone_start_idx].y, world_keypoints2[frame2_index][bone_start_idx].z])
        D = np.array([world_keypoints2[frame2_index][bone_finish_idx].x, world_keypoints2[frame2_index][bone_finish_idx].y, world_keypoints2[frame2_index][bone_finish_idx].z])
        bone_distance = get_distance_two_segments(A, B, C, D)
        if bone_distance > distance_threshold:
            highlight_indices.append(bone)



    image1 = draw_landmarks(frame1_7, keypoints1[frame1_index], (0, 255, 255))
    image2 = draw_landmarks(frame2_9, keypoints2[frame2_index], (0, 255, 0))
    
    
    for idx in highlight_indices:
        for idx_pair in highlight_indices:
            start_landmark = keypoints2[frame2_index][idx_pair[0]]
            end_landmark = keypoints2[frame2_index][idx_pair[1]]
            start_point = (int(start_landmark.x * image2.shape[1]), int(start_landmark.y * image2.shape[0]))
            end_point = (int(end_landmark.x * image2.shape[1]), int(end_landmark.y * image2.shape[0]))
            cv2.line(image2, start_point, end_point, (0, 0, 255), 5)

    combined_image = np.concatenate((image1, image2), axis=1)

    #cv2.putText(combined_image, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(combined_image, f'threshold: {round(distance_threshold,3)}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(combined_image, 'basic_vector', (1450, 200), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 255), 10, cv2.LINE_AA)
    cv2.imwrite(output_image, image2)

# 사용 예시
video_file1 = 'C:/Users/jk/action_assess_2/data/video/Z76/313-2-1-15-Z76_D.avi'
video_file2 = 'C:/Users/jk/action_assess_2/data/video/Z106/313-2-1-15-Z106_D.avi'
csv_file1 = 'C:/Users/jk/action_assess_2/data/csv/Z76/313-2-1-15-Z76_D.csv'
csv_file2 = 'C:/Users/jk/action_assess_2/data/csv/Z106/313-2-1-15-Z106_D.csv'
output_image = 'C:/Users/jk/action_assess_2/post_data/vector/output_image.png'
for threshold in np.arange(0.0, 0.12, 0.015):
    output_image = f'C:/Users/jk/action_assess_2/post_data/vector/basic/output_image_{round(threshold*1000)}.png'
    compare_vector_and_visualize(video_file1, video_file2, csv_file1, csv_file2, 4, 4, output_image, distance_threshold=threshold)

## world

In [95]:
import csv
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2


def read_keypoints_from_csv(csv_file):
    keypoints = []
    world_keypoints = []
    with open(csv_file, mode='r') as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            frame_keypoints = []
            frame_world_keypoints = []
            for i in range(33):
                x = float(row[1 + i*4])
                y = float(row[2 + i*4])
                z = float(row[3 + i*4])
                visibility = float(row[4 + i*4])
                frame_keypoints.append(landmark_pb2.NormalizedLandmark(x=x, y=y, z=z, visibility=visibility))
                
                world_x = float(row[133 + i*4])
                world_y = float(row[134 + i*4])
                world_z = float(row[135 + i*4])
                world_visibility = float(row[136 + i*4])
                frame_world_keypoints.append(landmark_pb2.NormalizedLandmark(x=world_x, y=world_y, z=world_z, visibility=world_visibility))
                
            keypoints.append(frame_keypoints)
            world_keypoints.append(frame_world_keypoints)
    return keypoints, world_keypoints

def extract_frame(video_path, frame_index):
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        raise ValueError(f"Could not read frame {frame_index} from {video_path}")
    return frame

def draw_landmarks(image, landmarks, color):
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    pose_landmarks = landmark_pb2.NormalizedLandmarkList(landmark=landmarks)
    mp_drawing.draw_landmarks(image, pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                                mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2))
    return image

def clamp(value, min_value, max_value):
    return max(min_value, min(value, max_value))

def get_closest_points_on_segments(A, B, C, D):
    """
    Calculate the closest points on two line segments in 3D space.
    
    Parameters:
    A, B, C, D (np.array): Points defining the two line segments (A-B and C-D).
    
    Returns:
    tuple: Closest points on the two line segments.
    """
    AB = B - A
    CD = D - C
    AC = A - C
    
    AB_dot_CD = np.dot(AB, CD)
    AB_dot_AB = np.dot(AB, AB)
    CD_dot_CD = np.dot(CD, CD)
    AC_dot_AB = np.dot(AC, AB)
    AC_dot_CD = np.dot(AC, CD)
    
    denom = AB_dot_AB * CD_dot_CD - AB_dot_CD * AB_dot_CD
    if denom == 0:
        # Segments are parallel, handle separately if needed
        return None, None
    
    s = (AB_dot_CD * AC_dot_CD - CD_dot_CD * AC_dot_AB) / denom
    t = (AB_dot_AB * AC_dot_CD - AB_dot_CD * AC_dot_AB) / denom
    
    # Clamp s and t to the range [0, 1] to ensure the points are on the segments
    s = clamp(s, 0, 1)
    t = clamp(t, 0, 1)
    
    P = A + s * AB
    Q = C + t * CD
    
    return P, Q

def get_distance_two_segments(A, B, C, D):
    """
    Calculate the shortest distance between two line segments in 3D space.
    
    Parameters:
    A, B, C, D (np.array): Points defining the two line segments (A-B and C-D).
    
    Returns:
    float: The shortest distance between the two line segments.
    """
    P, Q = get_closest_points_on_segments(A, B, C, D)
    if P is None or Q is None:
        # Segments are parallel, handle separately if needed
        return np.linalg.norm(np.cross(A - C, B - A)) / np.linalg.norm(B - A)
    
    return np.linalg.norm(P - Q)

def compare_vector_and_visualize(video_file1, video_file2, csv_file1, csv_file2, frame1_index, frame2_index, output_image, distance_threshold=0.04):
    keypoints1, world_keypoints1 = read_keypoints_from_csv(csv_file1)
    keypoints2, world_keypoints2 = read_keypoints_from_csv(csv_file2)

    frame1_7 = extract_frame(video_file1, frame1_index)
    frame2_9 = extract_frame(video_file2, frame2_index)

    highlight_indices = []
    for bone in observation_bones_list:

        bone_start_idx = bone[0]
        bone_finish_idx = bone[1]
        
        A = np.array([keypoints1[frame1_index][bone_start_idx].x, keypoints1[frame1_index][bone_start_idx].y, keypoints1[frame1_index][bone_start_idx].z])
        B = np.array([keypoints1[frame1_index][bone_finish_idx].x, keypoints1[frame1_index][bone_finish_idx].y, keypoints1[frame1_index][bone_finish_idx].z])
        
        C = np.array([keypoints2[frame2_index][bone_start_idx].x, keypoints2[frame2_index][bone_start_idx].y, keypoints2[frame2_index][bone_start_idx].z])
        D = np.array([keypoints2[frame2_index][bone_finish_idx].x, keypoints2[frame2_index][bone_finish_idx].y, keypoints2[frame2_index][bone_finish_idx].z])
        bone_distance = get_distance_two_segments(A, B, C, D)
        if bone_distance > distance_threshold:
            highlight_indices.append(bone)



    image1 = draw_landmarks(frame1_7, keypoints1[frame1_index], (0, 255, 255))
    image2 = draw_landmarks(frame2_9, keypoints2[frame2_index], (0, 255, 0))
    
    for idx in highlight_indices:
        for idx_pair in highlight_indices:
            start_landmark = keypoints2[frame2_index][idx_pair[0]]
            end_landmark = keypoints2[frame2_index][idx_pair[1]]
            start_point = (int(start_landmark.x * image2.shape[1]), int(start_landmark.y * image2.shape[0]))
            end_point = (int(end_landmark.x * image2.shape[1]), int(end_landmark.y * image2.shape[0]))
            cv2.line(image2, start_point, end_point, (0, 0, 255), 5)

    combined_image = np.concatenate((image1, image2), axis=1)

    #cv2.putText(combined_image, f'Score: {score}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(combined_image, f'threshold: {round(distance_threshold,3)}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(combined_image, 'world_vector', (1450, 200), cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 255), 10, cv2.LINE_AA)
    cv2.imwrite(output_image, combined_image)

# 사용 예시
video_file1 = 'C:/Users/jk/action_assess_2/data/video/Z76/313-2-1-15-Z76_D.avi'
video_file2 = 'C:/Users/jk/action_assess_2/data/video/Z106/313-2-1-15-Z106_D.avi'
csv_file1 = 'C:/Users/jk/action_assess_2/data/csv/Z76/313-2-1-15-Z76_D.csv'
csv_file2 = 'C:/Users/jk/action_assess_2/data/csv/Z106/313-2-1-15-Z106_D.csv'
output_image = 'C:/Users/jk/action_assess_2/post_data/vector/output_image.png'
for threshold in np.arange(0.0, 0.20, 0.005):
    output_image = f'C:/Users/jk/action_assess_2/post_data/vector/world/output_image_{round(threshold*1000)}.png'
    compare_vector_and_visualize(video_file1, video_file2, csv_file1, csv_file2, 4, 4, output_image, distance_threshold=threshold)

## appendix

In [108]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def clamp(value, min_value, max_value):
    return max(min_value, min(value, max_value))

def get_closest_points_on_segments(A, B, C, D):
    """
    Calculate the closest points on two line segments in 3D space.
    
    Parameters:
    A, B, C, D (np.array): Points defining the two line segments (A-B and C-D).
    
    Returns:
    tuple: Closest points on the two line segments.
    """
    AB = B - A
    CD = D - C
    AC = A - C
    
    AB_dot_CD = np.dot(AB, CD)
    AB_dot_AB = np.dot(AB, AB)
    CD_dot_CD = np.dot(CD, CD)
    AC_dot_AB = np.dot(AC, AB)
    AC_dot_CD = np.dot(AC, CD)
    
    denom = AB_dot_AB * CD_dot_CD - AB_dot_CD * AB_dot_CD
    if denom == 0:
        # Segments are parallel, handle separately if needed
        return None, None
    
    s = (AB_dot_CD * AC_dot_CD - CD_dot_CD * AC_dot_AB) / denom
    t = (AB_dot_AB * AC_dot_CD - AB_dot_CD * AC_dot_AB) / denom
    
    # Clamp s and t to the range [0, 1] to ensure the points are on the segments
    s = clamp(s, 0, 1)
    t = clamp(t, 0, 1)
    
    P = A + s * AB
    Q = C + t * CD
    
    return P, Q

def get_distance_two_segments(A, B, C, D):
    """
    Calculate the shortest distance between two line segments in 3D space.
    
    Parameters:
    A, B, C, D (np.array): Points defining the two line segments (A-B and C-D).
    
    Returns:
    float: The shortest distance between the two line segments.
    """
    P, Q = get_closest_points_on_segments(A, B, C, D)
    if P is None or Q is None:
        # Segments are parallel, handle separately if needed
        return np.linalg.norm(np.cross(A - C, B - A)) / np.linalg.norm(B - A)
    
    return np.linalg.norm(P - Q)

def plot_segments_and_points(A, B, C, D):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot points
    ax.scatter(*A, color='r', label='A')
    ax.scatter(*B, color='g', label='B')
    ax.scatter(*C, color='b', label='C')
    ax.scatter(*D, color='y', label='D')
    
    # Plot segments
    ax.plot(*zip(A, B), color='r', label='Segment AB')
    ax.plot(*zip(C, D), color='b', label='Segment CD')
    
    # Plot closest points and the line connecting them
    P, Q = get_closest_points_on_segments(A, B, C, D)
    if P is not None and Q is not None:
        ax.scatter(*P, color='m', label='Closest Point on AB')
        ax.scatter(*Q, color='c', label='Closest Point on CD')
        ax.plot(*zip(P, Q), color='k', linestyle='--', label='Shortest Distance')
    
    # Set labels
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    
    # Set legend
    ax.legend()
    
    plt.show()

# Example usage
A = np.array([3, 0, 1])
B = np.array([6, 5, 4])
C = np.array([0, -5, 0])
D = np.array([0, 10, 0])

distance = get_distance_two_segments(A, B, C, D)
print(f"The shortest distance between the segments is: {distance}")

plot_segments_and_points(A, B, C, D)

The shortest distance between the segments is: 4.594682917363407
